# Introduction to Signals and Systems:

This 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wave
import struct

# Parameters
sampling_rate = 44100  # Hz (CD quality)
duration = 2.0  # seconds
t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)

# Generate different waveforms
def generate_sine(freq, amplitude=0.5):
    return amplitude * np.sin(2 * np.pi * freq * t)

def generate_square(freq, amplitude=0.5):
    return amplitude * np.sign(np.sin(2 * np.pi * freq * t))

def generate_sawtooth(freq, amplitude=0.5):
    return amplitude * 2 * (t * freq - np.floor(t * freq + 0.5))

# Example 1: Pure sine wave (A440 - concert A)
freq_A = 440  # Hz
sine_wave = generate_sine(freq_A)

# Example 2: Chord (multiple frequencies)
C_major = generate_sine(261.63) + generate_sine(329.63) + generate_sine(392.00)
C_major = C_major / np.max(np.abs(C_major))  # Normalize

def save_wav(filename, signal, sample_rate=44100):
    # Normalize and convert to 16-bit PCM
    signal_normalized = np.int16(signal / np.max(np.abs(signal)) * 32767)
    
    with wave.open(filename, 'w') as wav_file:
        # Set parameters: nchannels, sampwidth, framerate, nframes, comptype, compname
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 2 bytes (16-bit)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(signal_normalized.tobytes())

# Save WAV files
save_wav('sine_wave.wav', sine_wave, sampling_rate)
save_wav('c_major.wav', C_major, sampling_rate)



In [12]:
# Note frequencies (in Hz) for C major scale
notes = {
    'C4': 261.63,  # Middle C (Do)
    'D4': 293.66,  # Re
    'E4': 329.63,  # Mi
    'F4': 349.23,  # Fa
    'G4': 392.00,  # Sol
    'A4': 440.00,  # La
    'B4': 493.88,  # Ti
    'C5': 523.25,  # High C (Do)
}

# Define chord structures (intervals from root)
chord_types = {
    'major': [0, 4, 7],      # Root, Major 3rd, Perfect 5th
    'minor': [0, 3, 7],      # Root, Minor 3rd, Perfect 5th
    '7': [0, 4, 7, 10],      # Major with minor 7th
    'maj7': [0, 4, 7, 11],   # Major with major 7th
    'add9': [0, 4, 7, 14],   # Major with added 9th (2 semitones = whole step above octave)
}
def get_note_freq(root, semitones_up):
    """Get frequency of a note N semitones above root"""
    return root * (2 ** (semitones_up / 12))

def generate_chord(root_note, chord_type, duration, notes = notes,
                   chord_types = chord_types, sample_rate=44100):
    """Generate a chord with specified root and type"""
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    root_freq = notes[root_note]
    
    # Generate each note in the chord
    chord_signal = np.zeros_like(t)
    intervals = chord_types[chord_type]
    
    for interval in intervals:
        freq = get_note_freq(root_freq, interval, notes = notes, chord_types = chord_types)
        chord_signal += np.sin(2 * np.pi * freq * t)
    
    # Normalize
    chord_signal = chord_signal / len(intervals)
    
    # Apply envelope (fade in/out to avoid clicks)
    envelope = np.ones_like(t)
    fade_samples = int(0.01 * sample_rate)  # 10ms fade
    envelope[:fade_samples] = np.linspace(0, 1, fade_samples)
    envelope[-fade_samples:] = np.linspace(1, 0, fade_samples)
    
    return chord_signal * envelope

def generate_note(frequency, duration, sample_rate=44100):
    """Generate a single note"""
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Generate sine wave
    note_signal = np.sin(2 * np.pi * frequency * t)
    
    # Apply envelope (fade in/out to avoid clicks)
    envelope = np.ones_like(t)
    fade_samples = int(0.01 * sample_rate)  # 10ms fade
    envelope[:fade_samples] = np.linspace(0, 1, fade_samples)
    envelope[-fade_samples:] = np.linspace(1, 0, fade_samples)
    
    return note_signal * envelope

def create_scale(note_list, note_duration=0.5, sample_rate=44100):
    """Create a scale from a list of notes"""
    full_scale = np.array([])
    
    for note_name in note_list:
        freq = notes[note_name]
        note = generate_note(freq, note_duration, sample_rate)
        full_scale = np.concatenate([full_scale, note])
    
    return full_scale

In [14]:
# C Major Scale: Do-Re-Mi-Fa-Sol-La-Ti-Do
scale_notes = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']
scale_labels = ['Do (C)', 'Re (D)', 'Mi (E)', 'Fa (F)', 'Sol (G)', 'La (A)', 'Ti (B)', 'Do (C)']

# Generate the scale
sample_rate = 44100
note_duration = 0.5  # seconds per note
scale = create_scale(scale_notes, note_duration, sample_rate)

def save_wav(filename, signal, sample_rate=44100):
    signal_normalized = np.int16(signal / np.max(np.abs(signal)) * 32767 * 0.8)
    with wave.open(filename, 'w') as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(signal_normalized.tobytes())

save_wav('c_major_scale.wav', scale, sample_rate)

In [ ]:
# Note frequencies (in Hz)
notes = {
    'C': 261.63, 'D': 293.66, 'E': 329.63, 'F': 349.23,
    'G': 392.00, 'A': 440.00, 'B': 493.88,
    'C#': 277.18, 'D#': 311.13, 'F#': 369.99, 'G#': 415.30, 'A#': 466.16
}





def create_chord_progression(progression, chord_duration=2.0, sample_rate=44100):
    """
    Create a sequence of chords
    progression: list of tuples (root_note, chord_type)
    """
    full_song = np.array([])
    
    for root, chord_type in progression:
        chord = generate_chord(root, chord_type, chord_duration, sample_rate)
        full_song = np.concatenate([full_song, chord])
    
    return full_song

my_progression = [
    ('C', 'major'),   # I
    ('G', 'major'),   # V
    ('A', 'minor'),   # vi
    ('F', 'major'),   # IV
    ('C', 'major'),   # I
    ('G', 'major'),   # V
    ('F', 'major'),   # IV
    ('F', 'major'),   # IV
]

# Generate the progression
sample_rate = 44100
chord_duration = 1.5  # seconds per chord
song = create_chord_progression(my_progression, chord_duration, sample_rate)

# Save to WAV file
def save_wav(filename, signal, sample_rate=44100):
    signal_normalized = np.int16(signal / np.max(np.abs(signal)) * 32767 * 0.8)
    with wave.open(filename, 'w') as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(signal_normalized.tobytes())

save_wav('chord_progression.wav', song, sample_rate)

In [ ]:
# Actual "Imagine" chord progression: C - Cmaj7 - F - F - Cadd9 - Cmaj7 - F - F
progression = [
    ('C', 'major'),
    ('C', 'maj7'),
    ('F', 'major'),
    ('F', 'major'),
    ('C', 'add9'),
    ('C', 'maj7'),
    ('F', 'major'),
    ('F', 'major'),
]

# Generate the progression
sample_rate = 44100
chord_duration = 2.0  # seconds per chord
song = create_chord_progression(progression, chord_duration, sample_rate)
def save_wav(filename, signal, sample_rate=44100):
    signal_normalized = np.int16(signal / np.max(np.abs(signal)) * 32767 * 0.8)
    with wave.open(filename, 'w') as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(signal_normalized.tobytes())

save_wav('imagine_chords.wav', song, sample_rate)